In [1]:
import json  
import requests 
import pandas as pd  
import matplotlib as plt 
import calendar
from googleapiclient.discovery import build  

In [2]:
CONST_USE_CACHE = True

In [3]:
api_key = pd.read_json('../config-master.json', typ='series')['gkey'] 
youtube = build("youtube", "v3", developerKey=api_key)  

# TRENDING CACHE

In [4]:
if not CONST_USE_CACHE:
    print('fetching api')
    datefrom = '2020-03-15'
    dateto = '2020-07-15'
    request = youtube.search().list(
        q="quarantine challenge",
        part=["snippet", "id"], 
        type="video", 
        publishedAfter=f"{datefrom}T00:00:00Z", 
        publishedBefore=f"{dateto}T00:00:00Z",
        videoDuration="any", 
        order="viewCount", 
        maxResults=50) 
    response=request.execute()  
    df=pd.DataFrame(response['items']) 
    df.to_json('SEARCH_CACHE.json') 

In [5]:
df=pd.DataFrame(pd.read_json('SEARCH_CACHE.json')) 
df_youtube_results=df.loc[:,['id','snippet']]
df=pd.merge(df, pd.json_normalize(df['id']), left_index=True, right_index=True) 
df=pd.merge(df, pd.json_normalize(df['snippet']), left_index=True, right_index=True) 
df=df.iloc[:,3:] 
df.drop([c for c in df.columns if 'thumb' in c], axis=1, inplace=True )  
df_youtube_results=df

request = youtube.videos().list( part='id,statistics' , id=list(df_youtube_results['videoId']))
response = request.execute()
response

# STATS CACHE

In [6]:
if not CONST_USE_CACHE:
    print('fetching api') 
    request = youtube.videos().list(
        part='id,statistics'
        , id=list(df_youtube_results['videoId']))
    print(request)
    response = request.execute()
    df=pd.DataFrame(response['items']) 
    df.to_json('VIDEO_CACHE.json') 

In [10]:
df=pd.DataFrame(pd.read_json('VIDEO_CACHE.json')) 
df=pd.merge(df, pd.json_normalize(df['statistics']), left_index=True, right_index=True) 
df=df.iloc[:,2:] 
df.drop([c for c in df.columns if 'statistics' in c], axis=1, inplace=True )  
df_stats=df

In [30]:
df=pd.merge(df_youtube_results, df_stats, left_on="videoId", right_on="id") 
df.drop([c for c in df.columns if ('_' in c) | (c in ['snippet', 'id'])], axis=1, inplace=True )    
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df['publishTime'] = pd.to_datetime(df['publishTime']) 
df_youtube=df
df_youtube.to_csv('youtube.csv') 

json=pd.read_json(f'SEARCH-2020-03-15.json')
df=pd.DataFrame(json)
dct = {}
for index, row in df.iterrows():
    print ( row[index]['id'])